In [1]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
import itertools
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import date
from os.path import join, isfile
from os import listdir
import time
from sklearn.metrics import f1_score
import tensorflow as tf 
from statistics import mode
import pickle5 as pickle

In [2]:
import pointnet_model

In [3]:
def read_rep(syn_name, syn_csv, ID_csv, data_dir):
    
    # open directories
    syn_dir = data_dir+"\\{}-patients".format(syn_name)
    ID_dir = data_dir+ "\\{}-selected-ID-controls".format(syn_name)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f))) and ".jpg" in f]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f))) and ".jpg" in f]
    
    data_syn, data_ID, labels_syn, labels_ID = [], [], [], []

    with open (syn_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_syn: 
                rep = list(map(float, row[1:]))
                data_syn.append(rep)
                labels_syn.append(1)

    with open (ID_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_ID:
                rep = list(map(float, row[1:]))
                data_ID.append(rep)
                labels_ID.append(0)

    return np.array(data_syn), np.array(data_ID), np.array(labels_syn), np.array(labels_ID)

In [4]:
def read_rep_landmarks(syn_name, syn_csv, ID_csv, data_dir):
    
    # open directories
    syn_dir = data_dir+"\\{}-patients".format(syn_name)
    ID_dir = data_dir+ "\\{}-selected-ID-controls".format(syn_name)

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f))) and ".jpg" in f]
    files_ID = [f for f in listdir(ID_dir) if (isfile(join(ID_dir, f))) and ".jpg" in f]

    data_syn, data_ID, labels_syn, labels_ID = [], [], [], []

    with open (syn_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_syn:
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append([float(row[i]), float(row[i+1]), float(row[i+2])])
                    i+=3                       
                data_syn.append(rep)
                labels_syn.append(1)

    with open (ID_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_ID:
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append([float(row[i]), float(row[i+1]), float(row[i+2])])
                    i+=3                       
                data_ID.append(rep)
                labels_ID.append(0)

    return np.array(data_syn), np.array(data_ID), np.array(labels_syn), np.array(labels_ID)

In [5]:
def normalize(data_1, data_2, data_3):
    data_1 = Normalizer().fit_transform(data_1)
    data_2 = Normalizer().fit_transform(data_2)
    data_3 = Normalizer().fit_transform(data_3)
    return data_1, data_2, data_3

In [6]:
def load_data(syn, GENERAL_DIR, data_dir): 

    method = "deepface"
    syn_csv = data_dir+"\\representations\{}-patients-{}.csv".format(syn, method)
    ID_csv  = data_dir+"\\representations\ID-controls-{}.csv".format(method)
    data_syn_df, data_ID_df, labels_syn_df, labels_ID_df = read_rep(syn, syn_csv, ID_csv, data_dir)
    
    method = "facereader-landmarks"
    syn_csv = GENERAL_DIR+ "\\features_facereader_landmarks_patient_groups.csv"
    ID_csv = GENERAL_DIR+ "\\features_facereader_landmarks_all_controls.csv" 
    data_syn_fr, data_ID_fr, _, _ = read_rep_landmarks(syn, syn_csv, ID_csv, data_dir)    
    
    method = "facereader-landmarks-distances"
    syn_csv = GENERAL_DIR+ "\\features_facereader_landmarks_distances_patient_groups_left_right.csv"
    ID_csv = GENERAL_DIR+ "\\features_facereader_landmarks_distances_all_controls_left_right.csv"    
    data_syn_dis, data_ID_dis, _,  _ = read_rep(syn, syn_csv, ID_csv, data_dir)  
    
    indices_to_keep = []
    for index, rep in enumerate(data_syn_dis):
        if not all(v == 0 for v in data_syn_dis[index]) and not all(v == 0 for v in data_ID_dis[index]):
            indices_to_keep.append(index)
                
    # all deepface data
    data_df = data_syn_df.tolist() + data_ID_df.tolist()
    labels_df = labels_syn_df.tolist() + labels_ID_df.tolist()
    
    # only deepface (that also has a facereader rep)
    data_syn_df_drop = data_syn_df[indices_to_keep]
    data_ID_df_drop = data_ID_df[indices_to_keep]
    data_df_drop = data_syn_df_drop.tolist() + data_ID_df_drop.tolist()
    
    # facereader landmarks 
    data_syn_fr = data_syn_fr[indices_to_keep]
    data_ID_fr = data_ID_fr[indices_to_keep]
    data_fr = data_syn_fr.tolist() + data_ID_fr.tolist()
    
    # only distance with facereader rep
    data_syn_dis = data_syn_dis[indices_to_keep]
    data_ID_dis = data_ID_dis[indices_to_keep]
    data_dis = data_syn_dis.tolist() + data_ID_dis.tolist()
    
    # labels with facereader rep
    labels_syn_df = labels_syn_df[indices_to_keep]
    labels_ID_df = labels_ID_df[indices_to_keep]
    labels = labels_syn_df.tolist() + labels_ID_df.tolist() 

    return np.array(data_df), np.array(data_df_drop), np.array(data_fr), np.array(data_dis), np.array(labels_df), np.array(labels)

In [7]:
def save_knn(syn, data, labels):        
    model = KNeighborsClassifier(n_neighbors=3, weights='distance')
    model.fit(data, labels)
    
    pickle.dump(model, open(r'C:\Users\manz616236\Documents\face-classification\web_application\models\knn-deepface-{}'.format(syn), 'wb'))

In [8]:
def save_pointnet(syn, data, labels):    
    model = pointnet_model.generate_model()
    model.fit(x=data, y=labels, batch_size=BATCH_SIZE, epochs=4, shuffle=True)
    path = r'C:\Users\manz616236\Documents\face-classification\web_application\models\pointnet-{}.ckpt'.format(syn)
    #tf.keras.models.save_model(model, ) 
    model.save_weights(path)

In [9]:
def save_randomforest(syn, data, labels):
    model = RandomForestClassifier(n_estimators=10)
    model.fit(data, labels)
    pickle.dump(model, open(r'C:\Users\manz616236\Documents\face-classification\web_application\models\randomforest-{}'.format(syn), 'wb'))

In [10]:
GENERAL_DIR = r"H:\Genetica Projecten\Facial Recognition\Studenten en Onderzoekers\Fien" 
BATCH_SIZE = 8
syn_list = ['ADNP', 'ANKRD11', 'CDK13', 'DEAF1', 'DYRK1A', 'EHMT1', 'FBXO11', 'KDVS', 'SON', 'WAC', 'YY1', '22q11'] 

# read in all data (per syndrome) which has a facereader and deepface representation
for syn in syn_list:
    
    data_dir = GENERAL_DIR + "\\{}".format(syn) 
    data_df_all, data_df, data_fr, data_dis, labels_df_all, labels = load_data(syn, GENERAL_DIR, data_dir)    
    data_df_all, data_df, data_dis = normalize(data_df_all, data_df, data_dis)   
    
    # DEEPFACE - KNN - all
    save_knn(syn, data_df_all, labels_df_all)
           
#     # POINTNET 
#     save_pointnet(syn, data_fr, labels)

#     # RANDOM FOREST 
#     save_randomforest(syn, data_dis, labels)

In [11]:
def read_rep_landmarks(syn_name, syn_csv, data_dir):
    
    # open directories
    syn_dir = data_dir+"\\{}-patients".format('ADNP')

    # get list of filenames
    files_syn = [f for f in listdir(syn_dir) if (isfile(join(syn_dir, f))) and ".jpg" in f]
    data_syn = []

    with open (syn_csv, newline='') as file:
        reader = csv.reader(file, delimiter=',')
        for index, row in enumerate(reader):
            if row[0] in files_syn:
                rep = []
                i = 1
                while i < len(row[1:]):
                    rep.append([float(row[i]), float(row[i+1]), float(row[i+2])])
                    i+=3                       
                data_syn.append(rep)

    return np.array(data_syn)

In [12]:
import time
print(time.time())

1597066092.8409507


## Load model and predict